In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()

True

In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
llm=HuggingFaceEndpoint(
    # repo_id="HuggingFaceH4/zephyr-7b-beta",
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    # repo_id="HuggingFaceH4/zephyr-7b-gemma-v0.1",
    # repo_id="lmsys/vicuna-13b-v1.5",
    task="text-generation"
)
model=ChatHuggingFace(llm=llm)
# generator=ChatHuggingFace(llm=llm)


In [3]:
class CrashState(TypedDict):
    input:str
    step1:str
    step2:str
    step3:str

In [4]:
def step1(state:CrashState):
    print("step1 Executed")
    return {
        "step1":"done",
        "input":state["input"]
    }

def step2(state:CrashState):
    print("Executing step 2....")
    import time
    time.sleep(30)
    return {
        "step2":"done"
    }

def step3(state:CrashState):
    print("step 3 done")
    return {
        "step3":"done"
    }

In [5]:
graph=StateGraph(CrashState)

graph.add_node('step1',step1)
graph.add_node('step2',step2)
graph.add_node('step3',step3)

graph.add_edge(START,"step1")
graph.add_edge("step1","step2")
graph.add_edge("step2","step3")
graph.add_edge("step3",END)

checkpointer=InMemorySaver()
workflow=graph.compile(checkpointer=checkpointer)

In [6]:
config={
    "configurable":{"thread_id":"1"}
}

workflow.invoke({"input":"Hello"},config=config)

step1 Executed
Executing step 2....


KeyboardInterrupt: 

In [7]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'input': 'Hello', 'step1': 'done'}, next=('step2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd32e-20e9-69ad-8001-99074f59d408'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-01-29T16:52:23.128499+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd32e-20e5-693a-8000-cbc7420694b6'}}, tasks=(PregelTask(id='60c71c67-50da-11f5-beab-3b4b25329477', name='step2', path=('__pregel_pull', 'step2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'Hello'}, next=('step1',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd32e-20e5-693a-8000-cbc7420694b6'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2026-01-29T16:52:23.126853+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd32e-20e2-6fb

FaultTolerance

In [ ]:
i=0
while i<5 and all_nodes_executed():
    workflow.invoke(None,config=config)
    i+=1


step1 Executed
Executing step 2....
step 3 done


{'input': 'Hello', 'step1': 'done', 'step2': 'done', 'step3': 'done'}